# Debug Runt Transform
template for testing/debugging runt transforms

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname
    
def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('runt_debug_template.ipynb', 'mutate' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numba import jit, vectorize
from dask import delayed, dataframe as dd

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

from common_util import MUTATE_DIR, DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ
from common_util import load_json, dump_json, remove_dups_list, list_get_dict, is_empty_df, search_df, benchmark
from data.data_api import DataAPI
from mutate.common import TRANSFORMS_DIR, get_graphs, get_transforms
from mutate.runt import process_transform

DEBUG:matplotlib:$HOME=/home/kev
DEBUG:matplotlib:CONFIGDIR=/home/kev/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /home/kev/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/kev/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.1.1
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltins', 'types', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib.util', 'impo

DEBUG:matplotlib:CACHEDIR=/home/kev/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/kev/.cache/matplotlib/fontlist-v310.json
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.
CRITICAL:root:script location: /home/kev/crunch/mutate/runt_debug_template.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


## General

### Init DataAPI, Set Logging Level, and Load Transforms

In [2]:
DataAPI.__init__()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
transforms = get_transforms()

## Transform Debug

### Set Transform
Fill it in (default is ddiff)

In [3]:
trf_name = 'ddiff'
step = transforms[trf_name]
display(step)

{'meta': {'rec_fmt': 'ddiff({num_periods})',
  'var_fmt': 'grid',
  'res_freq': 'D',
  'info': 'daily differenced data'},
 'fn': {'ser_fn': 'diff', 'df_fn': 'rut', 'col_fn': None, 'freq': 'D'},
 'axe': {'rm': None,
  'src': [['dohlca', 'dohlca'], ['dc', 'dc']],
  'dst': ['ddiff', 'ddiff']},
 'var': {'num_periods': [1]}}

In [4]:
display(step)

{'meta': {'rec_fmt': 'ddiff({num_periods})',
  'var_fmt': 'grid',
  'res_freq': 'D',
  'info': 'daily differenced data'},
 'fn': {'ser_fn': 'diff', 'df_fn': 'rut', 'col_fn': None, 'freq': 'D'},
 'axe': {'rm': None,
  'src': [['dohlca', 'dohlca'], ['dc', 'dc']],
  'dst': ['ddiff', 'ddiff']},
 'var': {'num_periods': [1]}}

### Test Section

In [5]:
runt_rcs, runt_dfs = process_transform(step, dump=False)

/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/home/kev/miniconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


INFO:root:data: ['dow_jones', 'dohlca', 'dohlca', 'pba_dohlca', 'pba_dohlca']
DEBUG:root:pre-runt:                            pba_open  pba_high   pba_low  pba_close  \
id                                                                   
2009-01-02 00:00:00+00:00   8771.77   9065.28   8761.10    9034.69   
2009-01-05 00:00:00+00:00   9027.13   9033.66   8892.68    8952.89   
2009-01-06 00:00:00+00:00   8954.57   9087.82   8941.11    9015.10   
2009-01-07 00:00:00+00:00   8996.94   8996.94   8720.00    8769.70   
2009-01-08 00:00:00+00:00   8769.94   8769.94   8651.27    8742.46   
...                             ...       ...       ...        ...   
2017-12-22 00:00:00+00:00  24764.04  24784.15  24717.51   24754.06   
2017-12-26 00:00:00+00:00  24715.84  24778.13  24708.42   24746.21   
2017-12-27 00:00:00+00:00  24766.52  24789.52  24731.68   24774.30   
2017-12-28 00:00:00+00:00  24807.21  24839.23  24797.13   24837.51   
2017-12-29 00:00:00+00:00  24849.63  24871.66  24719.22   247

DEBUG:root:pre-runt:                            etf3_buzz_S_open  etf3_buzz_S_high  \
id                                                              
2009-01-01 00:00:00+00:00      16825.100000      16825.100000   
2009-01-02 00:00:00+00:00      14513.500000      22962.900000   
2009-01-03 00:00:00+00:00      30348.500000      30348.500000   
2009-01-04 00:00:00+00:00      19750.200000      19750.200000   
2009-01-05 00:00:00+00:00      19267.500000      26952.100000   
...                                     ...               ...   
2017-12-27 00:00:00+00:00      20061.200000      20061.200000   
2017-12-28 00:00:00+00:00      19163.400000      19163.400000   
2017-12-29 00:00:00+00:00      18239.100001      20111.700001   
2017-12-30 00:00:00+00:00      17101.200001      17101.200001   
2017-12-31 00:00:00+00:00       9264.700001       9264.700001   

                           etf3_buzz_S_low  etf3_buzz_S_close  \
id                                                              
200

INFO:root:data: ['dow_jones', 'dohlca', 'dohlca', 'trmi3_mkt3_news_ratesBuzz_dohlca', 'trmi3_mkt3_news_ratesBuzz_dohlca']
DEBUG:root:pre-runt:                            mkt3_ratesBuzz_N_open  mkt3_ratesBuzz_N_high  \
id                                                                        
2009-01-01 00:00:00+00:00                 3018.0                 3018.0   
2009-01-02 00:00:00+00:00                  735.0                 1307.5   
2009-01-03 00:00:00+00:00                 1791.0                 1791.0   
2009-01-04 00:00:00+00:00                 1215.5                 1342.0   
2009-01-05 00:00:00+00:00                 1771.0                 2945.5   
...                                          ...                    ...   
2017-12-27 00:00:00+00:00                  678.5                  787.5   
2017-12-28 00:00:00+00:00                  991.5                 1010.0   
2017-12-29 00:00:00+00:00                  920.0                  951.0   
2017-12-30 00:00:00+00:00       

DEBUG:root:variant: {'num_periods': 1}
DEBUG:root:subvar: {'num_periods': 1}
DEBUG:root:mutate_type: ddiff(1)
DEBUG:root:post-runt:                            mkt3_stockIndexBuzz_N_open  \
id                                                      
2009-01-02 00:00:00+00:00               -13005.200000   
2009-01-03 00:00:00+00:00                 5183.600000   
2009-01-04 00:00:00+00:00                -6480.600000   
2009-01-05 00:00:00+00:00                 2413.900000   
2009-01-06 00:00:00+00:00                 4717.000000   
...                                               ...   
2017-12-27 00:00:00+00:00                 5760.600000   
2017-12-28 00:00:00+00:00                 1679.900000   
2017-12-29 00:00:00+00:00                 -221.000000   
2017-12-30 00:00:00+00:00                 1726.200001   
2017-12-31 00:00:00+00:00                -7853.400002   

                           mkt3_stockIndexBuzz_N_high  \
id                                                      
2009-01-02 0

...

DEBUG:root:variant: {'num_periods': 1}
DEBUG:root:subvar: {'num_periods': 1}
DEBUG:root:mutate_type: ddiff(1)
DEBUG:root:post-runt:                            mkt3_bondSentiment_N_close  \
id                                                      
2009-01-02 00:00:00+00:00                    0.047045   
2009-01-03 00:00:00+00:00                   -0.066399   
2009-01-04 00:00:00+00:00                    0.214477   
2009-01-05 00:00:00+00:00                   -0.064918   
2009-01-06 00:00:00+00:00                   -0.004857   
...                                               ...   
2017-12-27 00:00:00+00:00                    0.087140   
2017-12-28 00:00:00+00:00                   -0.127695   
2017-12-29 00:00:00+00:00                    0.035468   
2017-12-30 00:00:00+00:00                    0.053819   
2017-12-31 00:00:00+00:00                   -0.059830   

                           mkt3_bondOptimism_N_close  mkt3_bondTrust_N_close  \
id                                            

DEBUG:root:variant: {'num_periods': 1}
DEBUG:root:subvar: {'num_periods': 1}
DEBUG:root:mutate_type: ddiff(1)
DEBUG:root:post-runt:                            mkt3_stockIndexSentiment_N_close  \
id                                                            
2009-01-02 00:00:00+00:00                          0.050367   
2009-01-03 00:00:00+00:00                          0.051230   
2009-01-04 00:00:00+00:00                          0.229306   
2009-01-05 00:00:00+00:00                         -0.134373   
2009-01-06 00:00:00+00:00                         -0.102756   
...                                                     ...   
2017-12-27 00:00:00+00:00                          0.326047   
2017-12-28 00:00:00+00:00                          0.159663   
2017-12-29 00:00:00+00:00                         -0.057241   
2017-12-30 00:00:00+00:00                         -0.057566   
2017-12-31 00:00:00+00:00                          0.036815   

                           mkt3_stockIndexOptimi

INFO:root:data: ['sp_500', 'dc', 'dc', 'trmi3_mkt3_social_bank_dc', 'trmi3_mkt3_social_bank_dc']
DEBUG:root:pre-runt:                            mkt3_centralBank_S_close  mkt3_debtDefault_S_close  \
id                                                                              
2009-01-01 00:00:00+00:00                  0.539526                  0.353755   
2009-01-02 00:00:00+00:00                  0.624150                  0.236395   
2009-01-03 00:00:00+00:00                  0.640316                  0.239130   
2009-01-04 00:00:00+00:00                  0.502530                  0.342327   
2009-01-05 00:00:00+00:00                  0.647059                  0.217554   
...                                             ...                       ...   
2017-12-27 00:00:00+00:00                  0.619149                  0.193617   
2017-12-28 00:00:00+00:00                  0.543524                  0.326964   
2017-12-29 00:00:00+00:00                  0.636888                  0.1

DEBUG:root:variant: {'num_periods': 1}
DEBUG:root:subvar: {'num_periods': 1}
DEBUG:root:mutate_type: ddiff(1)
DEBUG:root:post-runt:                            mkt3_bondSentiment_S_close  \
id                                                      
2009-01-02 00:00:00+00:00                   -0.021224   
2009-01-03 00:00:00+00:00                   -0.032197   
2009-01-04 00:00:00+00:00                    0.074807   
2009-01-05 00:00:00+00:00                   -0.103842   
2009-01-06 00:00:00+00:00                    0.034205   
...                                               ...   
2017-12-27 00:00:00+00:00                   -0.104980   
2017-12-28 00:00:00+00:00                    0.135691   
2017-12-29 00:00:00+00:00                   -0.170664   
2017-12-30 00:00:00+00:00                    0.291621   
2017-12-31 00:00:00+00:00                   -0.291720   

                           mkt3_bondOptimism_S_close  mkt3_bondTrust_S_close  \
id                                            

DEBUG:root:variant: {'num_periods': 1}
DEBUG:root:subvar: {'num_periods': 1}
DEBUG:root:mutate_type: ddiff(1)
DEBUG:root:post-runt:                            mkt3_stockIndexSentiment_S_close  \
id                                                            
2009-01-02 00:00:00+00:00                          0.137311   
2009-01-03 00:00:00+00:00                         -0.053811   
2009-01-04 00:00:00+00:00                          0.109334   
2009-01-05 00:00:00+00:00                         -0.141362   
2009-01-06 00:00:00+00:00                         -0.035341   
...                                                     ...   
2017-12-27 00:00:00+00:00                          0.078505   
2017-12-28 00:00:00+00:00                          0.040468   
2017-12-29 00:00:00+00:00                          0.086204   
2017-12-30 00:00:00+00:00                         -0.095183   
2017-12-31 00:00:00+00:00                         -0.029727   

                           mkt3_stockIndexOptimi